Most # by station
\# by time frame
\# by line?

What are the busiest stations?
Where are people coming in? Where are people exiting?

Total by station?

What's the most busy data?

Further research:
    I would really like to know the gender of the people who are entering 

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import tkinter as tk
from tkinter import filedialog
from pandas import DataFrame
%matplotlib inline

# Set up the data
df = pd.read_csv('turnstile-usage-data-2018.csv', parse_dates=['Date'])

# Strip and make column names lowercase so that they are easy to manage. 
df.columns = df.columns.str.strip().str.lower()
# Replace characters or remove spaces and make all words lowercase.
df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_').str.replace('(', '').str.replace(')', '')

#Sort data such that each value 
df = df.sort_values(by=['station', 'scp', 'date', 'time'],ascending = [True, True, False, False])

# What's inside the file? Let's take a preview.
column_names = list(df.columns.values) 
print(column_names)
print(df.head())

#Be aware, the file is big so this process will take some time.

['c_a', 'unit', 'scp', 'station', 'line_name', 'division', 'date', 'time', 'description', 'entries', 'exits']
        c_a  unit       scp station line_name division       date      time  \
54926  H007  R248  00-00-00    1 AV         L      BMT 2018-12-28  23:00:00   
54929  H007  R248  00-00-00    1 AV         L      BMT 2018-12-28  23:00:00   
45501  H007  R248  00-00-00    1 AV         L      BMT 2018-12-28  19:00:00   
45503  H007  R248  00-00-00    1 AV         L      BMT 2018-12-28  19:00:00   
36049  H007  R248  00-00-00    1 AV         L      BMT 2018-12-28  15:00:00   

      description   entries     exits  
54926     REGULAR  14269744  15995511  
54929     REGULAR  14269744  15995511  
45501     REGULAR  14269286  15994982  
45503     REGULAR  14269286  15994982  
36049     REGULAR  14268317  15994262  


In [98]:
df.describe()
#We are still working with the full dataset so runtime is a bit high. We'll cut that down shortly.

,entries,exits
count,1.011147e+07,1.011147e+07
mean,3.961658e+07,3.258595e+07
std,2.058149e+08,1.904601e+08
min,0.000000e+00,0.000000e+00
25%,4.929705e+05,2.377950e+05
50%,2.527592e+06,1.426439e+06
75%,6.916714e+06,4.814437e+06
max,2.144893e+09,2.146093e+09


Clearly there is too much data here; we have here approximately 10 million data points to work with. Perhaps we can benefit from looking at a subset of the data. Let's take a slice of our data.

In [99]:
# Set ipython's max row display
# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

#List of unique values in the df['name'] column
for column_name in column_names:
    print("There are " + str(df[column_name].nunique()) + " unique values in column \'" + column_name + "\'.")

There are 749 unique values in column 'c_a'.
There are 470 unique values in column 'unit'.
There are 229 unique values in column 'scp'.
There are 379 unique values in column 'station'.
There are 114 unique values in column 'line_name'.
There are 6 unique values in column 'division'.
There are 350 unique values in column 'date'.
There are 85415 unique values in column 'time'.
There are 2 unique values in column 'description'.
There are 5701728 unique values in column 'entries'.
There are 5039144 unique values in column 'exits'.


In [100]:
#  Let's select the station that we're interested in.
station_name = "14 ST-UNION SQ"

# make new data frames to avoid hidden chaining
df_station = df[df.station == station_name].copy()
print(df_station.shape)

(81803, 11)


81803 values are a lot easier to deal with than 1.011147e+07
Let's get cracking...

In [101]:
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
canvas1.pack()

def exportCSV ():
    global df
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    df.to_csv (export_file_path, index = None, header=True)

saveAsButton_CSV = tk.Button(text='Export CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButton_CSV)

root.mainloop()

# We could also try:
# export_csv = df.to_csv (r'new_data_2018.csv', index = None, header=True)
# This is just a fancier way, using a GUI to save the file in a way that users will be familiar with.


In [102]:
#What's the data look like now?
df_station.head(40)

,c_a,unit,scp,station,line_name,division,date,time,description,entries,exits
49775,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,20:00:00,REGULAR,104986,232540
49780,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,20:00:00,REGULAR,104986,232540
40365,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,16:00:00,REGULAR,104906,232352
40370,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,16:00:00,REGULAR,104906,232352
30287,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,12:00:00,REGULAR,104834,232179
30290,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,12:00:00,REGULAR,104834,232179
18994,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,08:00:00,REGULAR,104787,232062
18995,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,08:00:00,REGULAR,104787,232062
9504,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,04:00:00,REGULAR,104780,232043
9507,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,04:00:00,REGULAR,104780,232043


In [103]:
df_station_cleaned = df_station.drop_duplicates().copy()
print(df_station.shape, df_station_cleaned.shape, df_station.shape[0] - df_station_cleaned.shape[0])

(81803, 11) (79966, 11) 1837


We removed 1837 duplicate values in this dataframe.

In [104]:
df_station_cleaned.head(40)

,c_a,unit,scp,station,line_name,division,date,time,description,entries,exits
49775,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,20:00:00,REGULAR,104986,232540
40365,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,16:00:00,REGULAR,104906,232352
30287,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,12:00:00,REGULAR,104834,232179
18994,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,08:00:00,REGULAR,104787,232062
9504,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,04:00:00,REGULAR,104780,232043
39,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,00:00:00,REGULAR,104776,232039
107292,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,20:00:00,REGULAR,104760,232004
97657,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,16:00:00,REGULAR,104650,231779
87840,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,12:00:00,REGULAR,104571,231654
78254,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,08:00:00,REGULAR,104535,231554


In [107]:
df_station_cleaned.loc[:, "entries_since_last_audit"] = df_station_cleaned.entries - df_station_cleaned.entries.shift(-1)
df_station_cleaned.loc[:, "exits_since_last_audit"] = df_station_cleaned.exits - df_station_cleaned.exits.shift(-1)
df_station_cleaned.head(20)
#df2 = df_station_cleaned.sort_values(by=['station', 'date', 'time', 'scp'],ascending = [True, False, False, True])
#df2.head(39)



,c_a,unit,scp,station,line_name,division,date,time,description,entries,exits,entries_since_last_audit,exits_since_last_audit,year
49775,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,20:00:00,REGULAR,104986,232540,80.0,188.0,2018
40365,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,16:00:00,REGULAR,104906,232352,72.0,173.0,2018
30287,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,12:00:00,REGULAR,104834,232179,47.0,117.0,2018
18994,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,08:00:00,REGULAR,104787,232062,7.0,19.0,2018
9504,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,04:00:00,REGULAR,104780,232043,4.0,4.0,2018
39,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-28,00:00:00,REGULAR,104776,232039,16.0,35.0,2018
107292,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,20:00:00,REGULAR,104760,232004,110.0,225.0,2018
97657,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,16:00:00,REGULAR,104650,231779,79.0,125.0,2018
87840,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,12:00:00,REGULAR,104571,231654,36.0,100.0,2018
78254,A035,R170,00-00-00,14 ST-UNION SQ,LNQR456W,BMT,2018-12-27,08:00:00,REGULAR,104535,231554,10.0,19.0,2018


In [ ]:
df_station_cleaned['year'] = df_station_cleaned['date'].dt.year.copy()
df_station_2018 = df_station_cleaned[df_station_cleaned.year == 2018].copy()
print(df_station_2018.shape, df_station_cleaned.shape, df_station_2018.shape[0] - df_station_cleaned.shape[0])
#gdate = df_station_cleaned.groupby('date')


We have removed 2061 non-2018 entries from this dataframe.

In [106]:
gdate_entries = df_station_2018.groupby(["date", "time"]).entries_since_last_audit.sum().reset_index().copy()
gdate_exits = df_station_2018.groupby(["date", "time"]).exits_since_last_audit.sum().reset_index().copy()

#gdate = df_station_2018.groupby(["date", "time"]).amount.sum().reset_index().sort_values("amount", ascending=False)
#gdate = df_station_2018.groupby(['date','time'])
gdate = pd.concat([gdate_entries, gdate_exits.exits_since_last_audit], axis = 1).copy()
gdate = gdate.sort_values("date", ascending=False)
gdate.head(40)

,date,time,entries_since_last_audit,exits_since_last_audit
2133,2018-12-28,20:00:00,26018.0,18237.0
2132,2018-12-28,16:00:00,19063.0,18993.0
2131,2018-12-28,12:00:00,7774.0,16034.0
2130,2018-12-28,11:09:45,787.0,1268.0
2129,2018-12-28,08:00:00,1794.0,6150.0
2128,2018-12-28,04:00:00,2559.0,904.0
2127,2018-12-28,00:00:00,15888.0,6068.0
2126,2018-12-27,20:00:00,31168.0,19600.0
2125,2018-12-27,16:00:00,20794.0,21533.0
2124,2018-12-27,12:00:00,-125481.0,-70424.0


**PART 1

* What does your data contain and what is its background? 
This dataset is titled NYS Turnstile Data Usage and comes from the New York State Open Data project. It is maintained and updated by Kaggle and can be retrieved at this link: https://www.kaggle.com/new-york-state/nys-turnstile-usage-data/. The dataset we have executed our code on was downloaded on July 3rd, 2019.

As the title suggests, it contains the number of entries and exits of turnstiles from different stations. This data is organized my control area (C/A), unit, subunit-channel-position(SCP), line, division, date and time and is retrieved (usually) at 4 hr intervals. 

* Where does it come from? Why is it interesting or significant? 
A whole host of questions can be asked from this dataset, especially for determining what stations are most busy and in what direction (are people mostly leaving the station or mostly entering the station) and how that can fluxuate throughout the day.

* Conduct summary statistics and produce visualizations for the particular variables from the dataset that you will use.

* What variables does your data contain and what distributions do you think they have? 
Variables in my data are entries and exits and the rest of the columns allow us to arrange the data by place or time. 

* Does the data bear on the preliminary questions you wrote down? 

* What new questions might you answer? How does the data look when you plot it out?

**PART 2

* Ask three analytic questions and answer each one with a combination of statistics and visualizations. These analytic questions can focus on individuals behaviors or comparisons of the population.

In [ ]:
Which station had the most entries
Which day was the busiest in each station
What time was the busiest in a particular day

